In [1]:
!pip install nfstream pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.0/195.0 kB 11.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
from nfstream import NFStreamer

my_streamer = NFStreamer(
    source="/content/Doh_traffic.pcap",
    statistical_analysis=True,
    active_timeout=25200,
)

df = my_streamer.to_pandas(columns_to_anonymize=[])

print(df.shape)
df.head()


(4, 86)


,id,expiration_id,src_ip,src_mac,src_oui,src_port,dst_ip,dst_mac,dst_oui,dst_port,...,dst2src_fin_packets,application_name,application_category_name,application_is_guessed,application_confidence,requested_server_name,client_fingerprint,server_fingerprint,user_agent,content_type
0,0,0,192.168.20.191,00:0c:29:f7:7f:d4,00:0c:29,57500,176.103.130.130,00:c1:b1:14:eb:31,00:c1:b1,443,...,0,TLS.DoH_DoT,Network,0,6,dns.adguard.com,66918128f1b9b03303d77c6f2eefd128,15af977ce25de452b96affa2addb1036,NaN,NaN
1,1,0,192.168.20.191,00:0c:29:f7:7f:d4,00:0c:29,57506,176.103.130.130,00:c1:b1:14:eb:31,00:c1:b1,443,...,0,TLS.DoH_DoT,Network,0,6,dns.adguard.com,66918128f1b9b03303d77c6f2eefd128,15af977ce25de452b96affa2addb1036,NaN,NaN
2,2,0,192.168.20.191,00:0c:29:f7:7f:d4,00:0c:29,57560,176.103.130.130,00:c1:b1:14:eb:31,00:c1:b1,443,...,0,TLS.DoH_DoT,Network,0,6,dns.adguard.com,66918128f1b9b03303d77c6f2eefd128,15af977ce25de452b96affa2addb1036,NaN,NaN
3,3,0,192.168.20.191,00:0c:29:f7:7f:d4,00:0c:29,57572,176.103.130.130,00:c1:b1:14:eb:31,00:c1:b1,443,...,0,TLS.DoH_DoT,Network,0,6,dns.adguard.com,66918128f1b9b03303d77c6f2eefd128,15af977ce25de452b96affa2addb1036,NaN,NaN


In [4]:
df.columns


Index(['id', 'expiration_id', 'src_ip', 'src_mac', 'src_oui', 'src_port',
       'dst_ip', 'dst_mac', 'dst_oui', 'dst_port', 'protocol', 'ip_version',
       'vlan_id', 'tunnel_id', 'bidirectional_first_seen_ms',
       'bidirectional_last_seen_ms', 'bidirectional_duration_ms',
       'bidirectional_packets', 'bidirectional_bytes', 'src2dst_first_seen_ms',
       'src2dst_last_seen_ms', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_first_seen_ms', 'dst2src_last_seen_ms',
       'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'bidirectional_min_ps', 'bidirectional_mean_ps',
       'bidirectional_stddev_ps', 'bidirectional_max_ps', 'src2dst_min_ps',
       'src2dst_mean_ps', 'src2dst_stddev_ps', 'src2dst_max_ps',
       'dst2src_min_ps', 'dst2src_mean_ps', 'dst2src_stddev_ps',
       'dst2src_max_ps', 'bidirectional_min_piat_ms',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'src2dst

In [5]:
features_df = pd.DataFrame()

features_df["sent_bytes"] = df["src2dst_bytes"]
features_df["received_bytes"] = df["dst2src_bytes"]
features_df["sent_packets"] = df["src2dst_packets"]
features_df["received_packets"] = df["dst2src_packets"]

features_df["byte_ratio"] = df["dst2src_bytes"] / df["src2dst_bytes"]
features_df["packet_ratio"] = df["dst2src_packets"] / df["src2dst_packets"]

features_df["connection_duration_sec"] = df["bidirectional_duration_ms"] / 1000

features_df["avg_recv_pkt_size"] = df["dst2src_mean_ps"]
features_df["var_recv_pkt_size"] = df["dst2src_stddev_ps"] ** 2

features_df["avg_sent_pkt_size"] = df["src2dst_mean_ps"]
features_df["var_sent_pkt_size"] = df["src2dst_stddev_ps"] ** 2

features_df["min_recv_delay_ms"] = df["dst2src_min_piat_ms"]
features_df["avg_recv_delay_ms"] = df["dst2src_mean_piat_ms"]
features_df["max_recv_delay_ms"] = df["dst2src_max_piat_ms"]

features_df


,sent_bytes,received_bytes,sent_packets,received_packets,byte_ratio,packet_ratio,connection_duration_sec,avg_recv_pkt_size,var_recv_pkt_size,avg_sent_pkt_size,var_sent_pkt_size,min_recv_delay_ms,avg_recv_delay_ms,max_recv_delay_ms
0,12113,19178,96,85,1.583258,0.885417,18.995,225.623529,87723.689916,126.177083,6361.094627,0,158.964286,4879
1,2534,8354,20,21,3.296764,1.050000,17.756,397.809524,283279.261905,126.700000,14562.536842,0,99.500000,1806
2,12148,19087,96,91,1.571205,0.947917,37.684,209.747253,83312.346520,126.541667,6320.671930,0,418.400000,6943
3,2480,8439,19,21,3.402823,1.105263,5.932,401.857143,282222.428571,130.526316,15062.485380,0,295.200000,5250


In [6]:
features_df.to_csv("sample-features.csv", index=False)


In [9]:


files.download("sample-features.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>